In [2]:
import time
from klampt import *
from IPython.display import clear_output
import ipywidgets as widgets
import ipyklampt

world = WorldModel()
def make_frame_widget(name,size,opacity=1.0):
    x = world.makeRigidObject(name+"_x")
    y = world.makeRigidObject(name+"_y")
    z = world.makeRigidObject(name+"_z")
    x.geometry().loadFile("data/cube.off")
    y.geometry().loadFile("data/cube.off")
    z.geometry().loadFile("data/cube.off")
    w = size*0.1
    x.geometry().transform([size,0,0,0,w,0,0,0,w],[0,-w*0.5,-w*0.5])
    y.geometry().transform([w,0,0,0,size,0,0,0,w],[-w*0.5,0,-w*0.5])
    z.geometry().transform([w,0,0,0,w,0,0,0,size],[-w*0.5,-w*0.5,0])
    x.appearance().setColor(1,0,0,1)
    y.appearance().setColor(0,1,0,1)
    z.appearance().setColor(0,0,1,1)

make_frame_widget("start_frame",0.3,opacity=0.5)
make_frame_widget("goal_frame",0.3,opacity=0.5)
make_frame_widget("current_frame",0.6)

kvis = ipyklampt.KlamptWidget(world)
playback_widget = ipyklampt.Playback(kvis)

#If you'd like to show the print output, comment out this line
clear_output()
display(kvis)
controls = widgets.HBox([])
display(controls)
display(playback_widget)

#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

S2xhbXB0V2lkZ2V0KHNjZW5lPXt1J29iamVjdCc6IHt1J21hdHJpeCc6IFsxLCAwLCAwLCAwLCAwLCAwLCAtMSwgMCwgMCwgMSwgMCwgMCwgMCwgMCwgMCwgMV0sIHUndXVpZCc6IHUnYTNhMzjigKY=


HBox()

UGxheWJhY2soY2hpbGRyZW49KEhCb3goY2hpbGRyZW49KEJ1dHRvbihkZXNjcmlwdGlvbj11J1BsYXknLCBpY29uPXUncGxheScsIHN0eWxlPUJ1dHRvblN0eWxlKCksIHRvb2x0aXA9dSdTdGHigKY=


In [3]:
import math
from klampt.math import so3,se3,vectorops

def interpolate_linear(a,b,u):
    """Interpolates linearly in cartesian space between a and b."""
    return vectorops.madd(a,vectorops.sub(b,a),u)

def interpolate_euler_angles(ea,eb,u,convention='zyx'):
    """Interpolates between the two euler angles.
    TODO: The default implementation interpolates linearly.  Can you
    do better?
    """
    return interpolate_linear(ea,eb,u)

def euler_angle_to_rotation(ea,convention='zyx'):
    """Converts an euler angle representation to a rotation matrix.
    Can use arbitrary axes specified by the convention
    arguments (default is 'zyx', or roll-pitch-yaw euler angles).  Any
    3-letter combination of 'x', 'y', and 'z' are accepted.
    """
    axis_names_to_vectors = dict([('x',(1,0,0)),('y',(0,1,0)),('z',(0,0,1))])
    axis0,axis1,axis2=convention
    R0 = so3.rotation(axis_names_to_vectors[axis0],ea[0])
    R1 = so3.rotation(axis_names_to_vectors[axis1],ea[1])
    R2 = so3.rotation(axis_names_to_vectors[axis2],ea[2])
    return so3.mul(R0,so3.mul(R1,R2))

#TODO: play around with these euler angles -- they'll determine the start and end of the rotations
ea = [math.pi/4,0,0]
eb = [math.pi*7/4,0,0]

def do_interpolate(u):
    global ea,eb
    #linear interpolation with euler angles
    e = interpolate_euler_angles(ea,eb,u)
    return euler_angle_to_rotation(e)
    #TODO: at the end of Problem 4.2, comment out the 3 prior lines and
    #uncomment this one.
    #return so3.interpolate(euler_angle_to_rotation(ea),euler_angle_to_rotation(eb),u)


In [5]:
#run this cell to bind your code to the visualization playback window
from klampt.math import vectorops,so3

#TODO: play around with these euler angles
Ta,Tb = None,None
ttotal = 0
period = 3.0
    
def set_frame_widget_xform(name,T):
    world.rigidObject(name+"_x").setTransform(*T)
    world.rigidObject(name+"_y").setTransform(*T)
    world.rigidObject(name+"_z").setTransform(*T)
    
def update_interpolation(u):
    global Ta,Tb
    R = do_interpolate(u)
    t = interpolate_linear(Ta[1],Tb[1],u)
    return (R,t)

def boilerplate_start():
    global Ta,Tb,ttotal
    Ta = (euler_angle_to_rotation(ea),(-1,0,0.5))
    Tb = (euler_angle_to_rotation(eb),(1,0,0.5))
    ttotal = 0
    set_frame_widget_xform("start_frame",Ta)
    set_frame_widget_xform("goal_frame",Tb)
    set_frame_widget_xform("current_frame",Ta)
    kvis.update()

def boilerplate_advance():
    #interpolate with a period of 3 seconds
    global ttotal,period
    ttotal += 0.02
    u = (ttotal%period)/period
    T = update_interpolation(u)
    set_frame_widget_xform("current_frame",T)

def manual_update_cb(value):
    global ttotal
    global Ta,Tb
    Ta = (euler_angle_to_rotation(ea),(-1,0,0.5))
    Tb = (euler_angle_to_rotation(eb),(1,0,0.5))
    set_frame_widget_xform("start_frame",Ta)
    set_frame_widget_xform("goal_frame",Tb)
    u = (ttotal%period)/period
    T = update_interpolation(u)
    set_frame_widget_xform("current_frame",T)
    kvis.update()

def time_update_cb(change):
    global ttotal
    playback_widget.stop()
    u = change['new']
    ttotal = u*period
    T = update_interpolation(u)
    set_frame_widget_xform("current_frame",T)
    kvis.update()
    
start = ipyklampt.EditPoint(value=ea,min=[0,0,0],max=[math.pi*2]*3,labels=['yaw1','pitch1','roll1'],callback=manual_update_cb)
end = ipyklampt.EditPoint(value=eb,min=[0,0,0],max=[math.pi*2]*3,labels=['yaw2','pitch2','roll2'],callback=manual_update_cb)
time = widgets.FloatSlider(description="time",value=0,min=0,max=1,step=0.01)
time.observe(time_update_cb,'value')
controls.children = (start,end,time)
        
boilerplate_start()
playback_widget.quiet = False
playback_widget.reset = boilerplate_start
playback_widget.advance = boilerplate_advance

In [ ]:
#testing... run this cell to verify your code
selfTest()

# Answers to Problem 4.2

Use this space to answer the written questions posed in Problem 4.2.
